<h1>Fine-tuning BERT on the Wizards of Wikipedia dataset</h1>

In [9]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
from datasets import load_metric
import json

from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertTokenizer

In [28]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

In [29]:
with open('wizards-of-wikipedia-data-extraction/out.json', 'r') as json_file:
    raw_dataset = json.load(json_file)

In [30]:
tokenizer = AutoTokenizer.from_pretrained("../models/bert-base-cased-squad2/saves")
#tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)

In [34]:
print(raw_dataset['output'][0]['human_sent'])
print(tokenize_function([raw_dataset['output'][0]['human_sent'], raw_dataset['output'][0]['knowledge_sent']]))
#print(tokenize_function((raw_dataset['output'][0]['human_sent'], raw_dataset['output'][0]['knowledge_sent'])))
#print(tokenize_function(raw_dataset['output'][0]))
#print(tokenize_function(raw_dataset['output'][0]))

I think science fiction is an amazing genre for anything. Future science, technology, time travel, FTL travel, they're all such interesting concepts.
{'input_ids': [[101, 146, 1341, 2598, 4211, 1110, 1126, 6929, 6453, 1111, 1625, 119, 7549, 2598, 117, 2815, 117, 1159, 3201, 117, 143, 20156, 3201, 117, 1152, 112, 1231, 1155, 1216, 5426, 8550, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
small_train_dataset = tokenized_datasets["output"].shuffle(seed=42).select(range(1000))

<h3>Approach #2: BERT2BERT</h3>

In [8]:
# leverage checkpoints for Bert2Bert model...
# use BERT's cls token as BOS token and sep token as EOS token
encoder = BertGenerationEncoder.from_pretrained("../models/bert-base-cased-squad2/saves", bos_token_id=101, eos_token_id=102)
# add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
decoder = BertGenerationDecoder.from_pretrained("../models/bert-base-cased-squad2/saves", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102)
bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)

# create tokenizer...
tokenizer = BertTokenizer.from_pretrained("../models/bert-base-cased-squad2/saves")

input_ids = tokenizer('This is a long article to summarize', add_special_tokens=False, return_tensors="pt").input_ids
labels = tokenizer('This is a short summary', return_tensors="pt").input_ids

# train...
loss = bert2bert(input_ids=input_ids, decoder_input_ids=labels, labels=labels).loss
loss.backward()

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ../models/bert-base-cased-squad2/saves were not used when initializing BertGenerationEncoder: ['bert.embeddings.token_type_embeddings.weight', 'qa_outputs.weight', 'qa_outputs.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertGenerationEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertGenerationEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You are using a model of type bert to instantiate a model of type bert-generation. Thi